<a href="https://colab.research.google.com/github/AtharvBhat/PIAYN/blob/main/PIAYN_TextClassificationLRABaseline_New_3k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 7.4 MB/s 
     |████████████████████████████████| 596 kB 58.5 MB/s 
     |████████████████████████████████| 77 kB 7.7 MB/s 
     |████████████████████████████████| 6.6 MB 52.9 MB/s 


In [2]:
!pip install -q datasets

     |████████████████████████████████| 325 kB 7.9 MB/s 
     |████████████████████████████████| 1.1 MB 60.2 MB/s 
     |████████████████████████████████| 212 kB 58.4 MB/s 
     |████████████████████████████████| 136 kB 50.3 MB/s 
     |████████████████████████████████| 127 kB 55.8 MB/s 
     |████████████████████████████████| 144 kB 11.4 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 271 kB 56.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
!pip install ml_collections
!pip install absl-py

     |████████████████████████████████| 77 kB 4.6 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=3a31c7191a81fd83fb00a548de6e53542135c73f72e7fabf342c05a84215a619
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
Successfully built ml-collections


In [4]:
!git clone https://github.com/google-research/long-range-arena.git
!git clone https://github.com/mlpen/Nystromformer.git
!git clone https://github.com/AtharvBhat/PIAYN.git

Cloning into 'long-range-arena'...
remote: Enumerating objects: 474, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 474 (delta 330), reused 423 (delta 282), pack-reused 0
Receiving objects: 100% (474/474), 153.28 KiB | 8.51 MiB/s, done.
Resolving deltas: 100% (330/330), done.
Cloning into 'Nystromformer'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 292 (delta 140), reused 147 (delta 42), pack-reused 0
Receiving objects: 100% (292/292), 10.82 MiB | 22.61 MiB/s, done.
Resolving deltas: 100% (140/140), done.
Cloning into 'PIAYN'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 6), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (22/22), done.


## Set up output folders

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Set the task
task = 'text'

#Set Seq Length
max_length = 3000

In [7]:
import os

baseDir = "/content/drive/MyDrive"
piaynDir = os.path.join(baseDir, "PIAYN")

if not os.path.exists(piaynDir):
  os.makedirs(piaynDir)

piaynTaskDir = os.path.join(piaynDir, task)

if not os.path.exists(piaynTaskDir):
  os.makedirs(piaynTaskDir)

piaynTaskModelDir = os.path.join(piaynTaskDir, "models")

if not os.path.exists(piaynTaskModelDir):
  os.makedirs(piaynTaskModelDir)

piaynTaskDataDir = os.path.join(piaynTaskDir, "data")

if not os.path.exists(piaynTaskDataDir):
  os.makedirs(piaynTaskDataDir)    

piaynTaskDataDirTokens = os.path.join(piaynTaskDataDir, str(max_length))

if not os.path.exists(piaynTaskDataDirTokens):
  os.makedirs(piaynTaskDataDirTokens)   

## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [8]:
from datasets import load_dataset
#from Nystromformer.LRA.datasets import text
import pickle, numpy as np

#copy over the custom text.py file to the Nystromformer location
!cp PIAYN/text.py Nystromformer/LRA/datasets

#Create Train test and dev MAP files for the imdb dataset
!python Nystromformer/LRA/datasets/text.py --max_length=$max_length

train_file = task+'.train.pickle'
dev_file = task+'.dev.pickle'
test_file = task+'.test.pickle'

!cp $train_file $piaynTaskDataDirTokens
!cp $dev_file $piaynTaskDataDirTokens
!cp $test_file $piaynTaskDataDirTokens

max_length:  3000
I0413 22:00:25.229904 139896763737984 dataset_builder.py:811] No config specified, defaulting to first: imdb_reviews/plain_text
2022-04-13 22:00:25.358317: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "NOT_FOUND: Error executing an HTTP request: HTTP response code 404".
I0413 22:00:25.585377 139896763737984 dataset_info.py:434] Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: imdb_reviews/plain_text/1.0.0
I0413 22:00:25.878839 139896763737984 dataset_info.py:361] Load dataset info from /tmp/tmpydc91gzztfds
I0413 22:00:25.880978 139896763737984 dataset_info.py:405] Field info.config_name from disk and from code do not match. Keeping the one from code.
I0413 22:00:25.881126 139896763737984 dataset_info.py:405] Fie

In [9]:
from Nystromformer.LRA.code import lra_config
from Nystromformer.LRA.code.dataset import LRADataset
#from Nystromformer.LRA.code.run_tasks import training_config
from torch.utils.data import DataLoader, RandomSampler

#get training config
training_config = lra_config.config[task]["training"]

#Update training config
#training_config["learning_rate"] = 0.00001
training_config["weight_decay"] = 0.1
training_config["eval_frequency"] = 1000

#Check Train Config
print('Training Config: ', training_config)

#get pre-defined model config
model_config = lra_config.config[task]['model']

#Check model Config
print('Model Config: ', model_config)

#Get the dataset
train_dataset = LRADataset(piaynTaskDataDirTokens + '/' + train_file, False)
val_dataset = LRADataset(piaynTaskDataDirTokens + '/' + dev_file, False)
test_dataset = LRADataset(piaynTaskDataDirTokens + '/' + test_file, False)

#Create DataLoader iterators
ds_iter = {
    "train":enumerate(DataLoader(train_dataset, 
                                 #Sample batches randomly for number of specified steps
                                 sampler = RandomSampler(train_dataset, 
                                                         replacement=True, 
                                                         num_samples= training_config["num_train_steps"]*lra_config.config[task]['dataset']['train']), 
                                 batch_size = training_config["batch_size"], 
                                 drop_last = True)),
    "dev":enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True)),
    "test":enumerate(DataLoader(test_dataset, batch_size = 32, drop_last = True)),
}


Training Config:  {'batch_size': 32, 'learning_rate': 0.0001, 'warmup': 8000, 'lr_decay': 'linear', 'weight_decay': 0.1, 'eval_frequency': 1000, 'num_train_steps': 20000, 'num_eval_steps': 781}
Model Config:  {'learn_pos_emb': True, 'tied_weights': False, 'embedding_dim': 64, 'transformer_dim': 64, 'transformer_hidden_dim': 128, 'head_dim': 32, 'num_head': 2, 'num_layers': 2, 'vocab_size': 512, 'max_seq_len': 4000, 'dropout_prob': 0.1, 'attention_dropout': 0.1, 'pooling_mode': 'MEAN', 'num_classes': 2}
Loaded /content/drive/MyDrive/PIAYN/text/data/3000/text.train.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/3000/text.dev.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/3000/text.test.pickle... size=25000


In [10]:
#Check sizes of batches
batch = next((ds_iter['train']))
for k,v in batch[1].items():
  print(k,v.shape)

input_ids_0 torch.Size([32, 3024])
mask_0 torch.Size([32, 3024])
label torch.Size([32])


## Define model

Next, we define our model, and put it on the GPU.

In [11]:
from transformers import PerceiverForSequenceClassification
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
from transformers import PerceiverConfig
#get default perceiver config
configuration = PerceiverConfig()

#Update the Perceiver configurations with Preset model configs
#configuration.update(model_config)

#Print Updated Perceiver Configuration
print(configuration)

PerceiverConfig {
  "attention_probs_dropout_prob": 0.1,
  "audio_samples_per_frame": 1920,
  "cross_attention_shape_for_attention": "kv",
  "cross_attention_widening_factor": 1,
  "d_latents": 1280,
  "d_model": 768,
  "hidden_act": "gelu",
  "image_size": 56,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "perceiver",
  "num_blocks": 1,
  "num_cross_attention_heads": 8,
  "num_frames": 16,
  "num_latents": 256,
  "num_self_attends_per_block": 26,
  "num_self_attention_heads": 8,
  "output_shape": [
    1,
    16,
    224,
    224
  ],
  "qk_channels": null,
  "samples_per_patch": 16,
  "self_attention_widening_factor": 1,
  "train_size": [
    368,
    496
  ],
  "transformers_version": "4.19.0.dev0",
  "use_query_residual": true,
  "v_channels": null,
  "vocab_size": 262
}



In [13]:
def initialize_model(config):
  #Initialize Model
  model = PerceiverForSequenceClassification(config)
  
  #Get Model Parameter Counts
  pytorch_total_params = sum(p.numel() for p in model.parameters())
  pytorch_total_params_Trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print('Total Parameters: ', pytorch_total_params, '\nTrainable Parameters: ', pytorch_total_params_Trainable)  

  return model#.to(device)                                          

In [14]:
#Change the perceiver configurations to get total parameters within 10% of BERT
configuration.num_labels = 2
configuration.num_self_attends_per_block = 6
configuration.d_latents = 512
configuration.d_model = 512
configuration.num_layers = 6
configuration.max_position_embeddings = max_length + 24*np.power(2, int(max_length/1000)-1)



model = initialize_model(configuration)

Total Parameters:  14477826 
Trainable Parameters:  14477826


## Train the model

Here we train the model using native PyTorch.

In [15]:
device

device(type='cuda')

In [16]:
from transformers import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from datasets import load_metric
import pandas as pd

best_score = 0 
prev_score = 0
maxPatience = 3
currentPatience = 0

#steps = int(training_config["num_train_steps"]/20000)
steps = training_config["num_train_steps"]

optimizer = AdamW(model.parameters(), 
                  lr = training_config["learning_rate"],
                  betas = (0.9, 0.999), 
                  eps = 1e-6, 
                  weight_decay = training_config["weight_decay"])

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer = optimizer,
    max_lr = training_config["learning_rate"],
    pct_start = training_config["warmup"] / training_config["num_train_steps"],
    anneal_strategy = training_config["lr_decay"],
    total_steps = training_config["num_train_steps"]
)

#amp_scaler = torch.cuda.amp.GradScaler() if model_config["mixed_precision"] else None

#initialize training summary
trainingSummary = pd.DataFrame(columns=['step', 'train_acc', 'val_acc'])


model.to(device)

#initialize training accuracy metric
train_accuracy = load_metric("accuracy")

for step in tqdm(range(steps)):  # Perform gradient updates for multiple steps
    
    model.train()
    
    #print("Step:", step)
    #for batch in tqdm(train_dataloader):
    batch = next(ds_iter['train'])[1]

    # get the inputs; 
    inputs = batch["input_ids_0"].to(device)
    attention_mask = batch["mask_0"].to(device)
    labels = batch["label"].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs=inputs, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # evaluate
    predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
    accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
    references = batch["label"].numpy()
    train_accuracy.add_batch(predictions=predictions, references=references)
    
    print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

    #delete intermediate variables to free up GPU space
    del loss, outputs, inputs, attention_mask, labels, predictions, accuracy


    #Every 1000 steps validate and save model
    if (step+1)%training_config['eval_frequency']  == 0:
    #if (step+1)%2  == 0:
      
      model.eval()

      val_accuracy = load_metric("accuracy")

      #reset dev iterator
      ds_iter['dev'] = enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True))

      with torch.no_grad():
        for i, batch in tqdm(ds_iter['dev']):
              
          # get the inputs; 
          inputs = batch["input_ids_0"].to(device)
          attention_mask = batch["mask_0"].to(device)
          labels = batch["label"].to(device)

          #print('input shape: ', inputs.shape)
          #print('attention_mask shape: ', attention_mask.shape)
          #print('labels: ', labels.shape)

          # forward pass
          outputs = model(inputs=inputs, attention_mask=attention_mask)
          logits = outputs.logits 
          predictions = logits.argmax(-1).cpu().detach().numpy()
          references = batch["label"].numpy()
          val_accuracy.add_batch(predictions=predictions, references=references)

          #delete intermediate variables to free up GPU space
          del logits, outputs, inputs, attention_mask, labels, predictions, references
      
      #Compute val accuracy
      final_val_score = val_accuracy.compute()['accuracy']
      print("Validation Accuracy:", final_val_score)

      if final_val_score >= best_score:
        best_score = final_val_score
        torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainedPerceiverClassifierToken'+str(max_length)+'.pkl')
        model.to(device)
      else:
        pass  

      if final_val_score <= prev_score:
        currentPatience += 1
        if currentPatience >= maxPatience:
          print('Patience Limit reached! Stopping early!')
          torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainedPerceiverClassifierStep_' + str(step + 1) + 'Token' + str(max_length) + '.pkl')
          break  
      else:
        currentPatience = 0
      
      #Update prev_score
      prev_score = final_val_score

      #Compute training accuracy till now
      final_train_score = train_accuracy.compute()['accuracy']

      #Add to trainingSummary
      trainingSummary.loc[len(trainingSummary.index)] = [step+1, final_train_score, final_val_score]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/20000 [00:00<?, ?it/s]

Loss: 0.6855388283729553, Accuracy: 0.5625
Loss: 0.7097299098968506, Accuracy: 0.46875
Loss: 0.7095855474472046, Accuracy: 0.4375
Loss: 0.6905152201652527, Accuracy: 0.53125
Loss: 0.6943463683128357, Accuracy: 0.5
Loss: 0.6933883428573608, Accuracy: 0.46875
Loss: 0.6988707780838013, Accuracy: 0.46875
Loss: 0.6898512244224548, Accuracy: 0.53125
Loss: 0.7072557806968689, Accuracy: 0.4375
Loss: 0.7223803400993347, Accuracy: 0.375
Loss: 0.6818923354148865, Accuracy: 0.59375
Loss: 0.7058872580528259, Accuracy: 0.40625
Loss: 0.6899739503860474, Accuracy: 0.5
Loss: 0.6913514733314514, Accuracy: 0.5
Loss: 0.6869654655456543, Accuracy: 0.65625
Loss: 0.691534698009491, Accuracy: 0.5
Loss: 0.7048271298408508, Accuracy: 0.46875
Loss: 0.7347606420516968, Accuracy: 0.34375
Loss: 0.673775315284729, Accuracy: 0.59375
Loss: 0.7201433777809143, Accuracy: 0.40625
Loss: 0.7391360998153687, Accuracy: 0.34375
Loss: 0.7040286660194397, Accuracy: 0.46875
Loss: 0.6853067874908447, Accuracy: 0.5625
Loss: 0.7085

0it [00:00, ?it/s]

Validation Accuracy: 0.6071142765685019
Loss: 0.7794294953346252, Accuracy: 0.46875
Loss: 0.5437597632408142, Accuracy: 0.75
Loss: 0.6907886862754822, Accuracy: 0.5
Loss: 0.6851173639297485, Accuracy: 0.625
Loss: 0.6096206903457642, Accuracy: 0.65625
Loss: 0.5974438190460205, Accuracy: 0.65625
Loss: 0.6228463053703308, Accuracy: 0.6875
Loss: 0.704923689365387, Accuracy: 0.53125
Loss: 0.6543349027633667, Accuracy: 0.6875
Loss: 0.5528475046157837, Accuracy: 0.71875
Loss: 0.6319539546966553, Accuracy: 0.65625
Loss: 0.6383905410766602, Accuracy: 0.625
Loss: 0.719225287437439, Accuracy: 0.5625
Loss: 0.6023035645484924, Accuracy: 0.71875
Loss: 0.6675892472267151, Accuracy: 0.625
Loss: 0.6012824773788452, Accuracy: 0.65625
Loss: 0.7150207757949829, Accuracy: 0.5625
Loss: 0.6422348022460938, Accuracy: 0.65625
Loss: 0.7277967929840088, Accuracy: 0.53125
Loss: 0.6713859438896179, Accuracy: 0.5625
Loss: 0.6229020953178406, Accuracy: 0.65625
Loss: 0.701026201248169, Accuracy: 0.5625
Loss: 0.604211

...

Validation Accuracy: 0.6419654289372599
Loss: 0.630294919013977, Accuracy: 0.71875
Loss: 0.4372868239879608, Accuracy: 0.75
Loss: 0.4734865128993988, Accuracy: 0.71875
Loss: 0.6303135752677917, Accuracy: 0.6875
Loss: 0.6123724579811096, Accuracy: 0.6875
Loss: 0.6442545652389526, Accuracy: 0.625
Loss: 0.6445144414901733, Accuracy: 0.625
Loss: 0.5310454368591309, Accuracy: 0.8125
Loss: 0.6977230906486511, Accuracy: 0.53125
Loss: 0.6314614415168762, Accuracy: 0.71875
Loss: 0.6551442742347717, Accuracy: 0.65625
Loss: 0.37478145956993103, Accuracy: 0.9375
Loss: 0.4532754123210907, Accuracy: 0.84375
Loss: 0.5529061555862427, Accuracy: 0.75
Loss: 0.6047120094299316, Accuracy: 0.71875
Loss: 0.5604321360588074, Accuracy: 0.65625
Loss: 0.5400897264480591, Accuracy: 0.75
Loss: 0.6189422011375427, Accuracy: 0.625
Loss: 0.6079021096229553, Accuracy: 0.625
Loss: 0.6439116597175598, Accuracy: 0.5625
Loss: 0.4526499807834625, Accuracy: 0.84375
Loss: 0.5313994288444519, Accuracy: 0.71875
Loss: 0.457207

0it [00:00, ?it/s]

Validation Accuracy: 0.6354433418693982
Loss: 0.5159416794776917, Accuracy: 0.75
Loss: 0.590811550617218, Accuracy: 0.71875
Loss: 0.6240846514701843, Accuracy: 0.625
Loss: 0.6759335398674011, Accuracy: 0.625
Loss: 0.5613991022109985, Accuracy: 0.65625
Loss: 0.5058547258377075, Accuracy: 0.84375
Loss: 0.5045449733734131, Accuracy: 0.78125
Loss: 0.5318865180015564, Accuracy: 0.78125
Loss: 0.4055708646774292, Accuracy: 0.84375
Loss: 0.47177422046661377, Accuracy: 0.78125
Loss: 0.46632567048072815, Accuracy: 0.8125
Loss: 0.5985393524169922, Accuracy: 0.71875
Loss: 0.6413007974624634, Accuracy: 0.625
Loss: 0.38517430424690247, Accuracy: 0.84375
Loss: 0.5696595907211304, Accuracy: 0.6875
Loss: 0.3956509828567505, Accuracy: 0.75
Loss: 0.5472126007080078, Accuracy: 0.78125
Loss: 0.3948274552822113, Accuracy: 0.78125
Loss: 0.4970896244049072, Accuracy: 0.6875
Loss: 0.6032049655914307, Accuracy: 0.8125
Loss: 0.5961878895759583, Accuracy: 0.71875
Loss: 0.5879570841789246, Accuracy: 0.71875
Loss: 

0it [00:00, ?it/s]

Validation Accuracy: 0.6193181818181818
Patience Limit reached! Stopping early!


In [17]:
#save training summary
trainingSummary.to_csv(piaynTaskModelDir + '/trainingSummaryToken'+str(max_length)+'.csv')

In [18]:
!nvidia-smi

Wed Apr 13 22:47:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    50W / 250W |   8697MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
del model
torch.cuda.empty_cache()
!nvidia-smi

Wed Apr 13 22:47:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P0    50W / 250W |   1261MiB / 16280MiB |     24%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Evaluate the model

Finally, we evaluate the model on the test set. We use the Datasets library to compute the accuracy.

In [20]:
from tqdm.notebook import tqdm
from datasets import load_metric

accuracy = load_metric("accuracy")

#load best performing model checkpoint
model = PerceiverForSequenceClassification(configuration)
model.load_state_dict(torch.load(piaynTaskModelDir + '/trainedPerceiverClassifierToken'+str(max_length)+'.pkl'))
model.to(device)

model.eval()

with torch.no_grad():
  for i, batch in tqdm(ds_iter['test']):
        
        # get the inputs; 
        inputs = batch["input_ids_0"].to(device)
        attention_mask = batch["mask_0"].to(device)
        labels = batch["label"].to(device)

        # forward pass
        outputs = model(inputs=inputs, attention_mask=attention_mask)
        logits = outputs.logits 
        predictions = logits.argmax(-1).cpu().detach().numpy()
        references = batch["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)

        #delete intermediate variables to free up GPU space
        del logits, outputs, inputs, attention_mask, labels, predictions, references

final_score = accuracy.compute()
print("Accuracy on test set:", final_score['accuracy'])

0it [00:00, ?it/s]

Accuracy on test set: 0.6466869398207427
